---------------

## 1. Loading the datasets from CSVs
- ~~[데이터셋 링크](https://uwnlp.github.io/storycommonsense/) 에서 [download the data]를 클릭해 다운로드 해주세요.~~  
- ~~다운받은 storycommonsense_data.zip 파일을 업로드 해주세요.~~
- 위의 과정을 wget 명령어로 코드 줄에서 해결합니다. (다운로드->업로드 과정 생략)
- 아래 코드로 압축을 푼 다음, /content/csv_version 경로 안의 dev/emotion, test/emotion 안의 csv 파일을 합친 후 6:4 비율로 각각 나눠 train과 test 데이터셋으로 사용합니다. (training 폴더 안의 csv 파일은 plutchik의 8가지 감정을 분류되어 있지 않기 때문에, dev의 csv파일을 사용합니다.)


In [1]:
!wget https://uwnlp.github.io/storycommonsense/data/storycommonsense_data.zip 

--2020-12-10 13:41:07--  https://uwnlp.github.io/storycommonsense/data/storycommonsense_data.zip
Resolving uwnlp.github.io (uwnlp.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to uwnlp.github.io (uwnlp.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18755402 (18M) [application/zip]
Saving to: ‘storycommonsense_data.zip’

storycommonsense_da 100%[===================>]  17.89M  --.-KB/s    in 0.1s    

2020-12-10 13:41:07 (188 MB/s) - ‘storycommonsense_data.zip’ saved [18755402/18755402]



In [2]:
!unzip /content/storycommonsense_data.zip

Archive:  /content/storycommonsense_data.zip
   creating: csv_version/
   creating: csv_version/dev/
   creating: csv_version/dev/emotion/
  inflating: csv_version/dev/emotion/allcharlinepairs.csv  
   creating: csv_version/dev/motiv/
  inflating: csv_version/dev/motiv/allcharlinepairs.csv  
  inflating: csv_version/README.txt  
   creating: csv_version/test/
   creating: csv_version/test/emotion/
  inflating: csv_version/test/emotion/allcharlinepairs.csv  
   creating: csv_version/test/motiv/
  inflating: csv_version/test/motiv/allcharlinepairs.csv  
   creating: csv_version/training/
  inflating: csv_version/training/allcharlinepairs.csv  
   creating: csv_version/characters/
  inflating: csv_version/characters/entitylines_devtest.csv  
  inflating: csv_version/characters/entitylines_train.csv  
   creating: json_version/
  inflating: json_version/README.txt  
  inflating: json_version/annotations.json  
  inflating: README.txt              
  inflating: rocstorysubset.csv      
  in

In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('/content/csv_version/dev/emotion/allcharlinepairs.csv')
test = pd.read_csv('/content/csv_version/test/emotion/allcharlinepairs.csv')

In [5]:
train.head(3)

,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
0,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann0,NaN,I began making fish curry for my boyfriend and I.,yes,"[""Joy and excited to be making the food""]","[""joy:3"", ""trust:3"", ""surprise:3"", ""anticipati..."
1,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann1,NaN,I began making fish curry for my boyfriend and I.,yes,"[""content""]","[""joy:2""]"
2,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann2,NaN,I began making fish curry for my boyfriend and I.,yes,"[""hungry"", ""anticipation""]","[""joy:2"", ""anticipation:3""]"


In [6]:
test.head(3)

,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
0,a3e66276-ef27-426f-8a2f-329779f17038,1,Cook,ann0,NaN,A cook was carrying an armful of oranged in th...,yes,"[""joy""]","[""joy:3"", ""trust:3"", ""surprise:2"", ""anticipati..."
1,a3e66276-ef27-426f-8a2f-329779f17038,1,Cook,ann1,NaN,A cook was carrying an armful of oranged in th...,yes,"[""useful"", ""nervous""]","[""joy:2"", ""anticipation:2""]"
2,a3e66276-ef27-426f-8a2f-329779f17038,2,Cook,ann0,A cook was carrying an armful of oranged in th...,He dropped one on the floor by accident.,yes,"[""annoyed""]","[""fear:2"", ""surprise:2"", ""anticipation:2""]"


### 1.1 3명의 annotator가 태깅한 문장 추출
- setence : 문장
- ann0 : annotator A가 태깅한 해당 문장에서 느껴지는 감정 리스트
- ann1 : annotator B가 태깅한 해당 문장에서 느껴지는 감정 리스트
- ann2 : annotator C가 태깅한 해당 문장에서 느껴지는 감정 리스트

In [7]:
train = train.replace({'ann0':0,'ann1':1,'ann2':2})
train.drop(train.loc[(train['emotionworkerid']!=0)&(train['emotionworkerid']!=1)&(train['emotionworkerid']!=2)].index, inplace=True)
train = train.reset_index(drop=True)

test = test.replace({'ann0':0,'ann1':1,'ann2':2})
test.drop(test.loc[(test['emotionworkerid']!=0)&(test['emotionworkerid']!=1)&(test['emotionworkerid']!=2)].index, inplace=True)
test = test.reset_index(drop=True)

In [8]:
def extract_3_anno(df):
  ann0 = []
  ann1 = []
  ann2 = []
  sentence = []
  ann0_idx = []
  ann1_idx = []
  ann2_idx = []

  storyid = str()
  linenum = 0
  has0 = None
  has1 = None

  cnt = 0
  for idx in range(0,len(df)):
    if storyid != df['storyid'][idx]:
      storyid = df['storyid'][idx]
    if linenum != df['linenum'][idx]:
      linenum = df['linenum'][idx]
    
    if storyid == df['storyid'][idx] and linenum == df['linenum'][idx]:
      if df['emotionworkerid'][idx] == 0:
        df['emotionworkerid'][idx] == 0
        has0 = idx
      elif df['emotionworkerid'][idx] == 1:
        has1 = idx
      else:
        if has0 !=None and has1 !=None:
          ann0.append(df['plutchik'][has0])
          ann1.append(df['plutchik'][has1])
          ann2.append(df['plutchik'][idx])
          sentence.append(df['sentence'][idx])
          ann0_idx.append(has0)
          ann1_idx.append(has1)
          ann2_idx.append(idx)
        cnt+=1
    else:
      has0 = None
      has1 = None

  ann0_df = df.loc[ann0_idx]
  ann0_df = ann0_df.reset_index(drop=True)
  ann1_df = df.loc[ann1_idx]
  ann1_df = ann1_df.reset_index(drop=True)
  ann2_df = df.loc[ann2_idx]
  ann2_df = ann2_df.reset_index(drop=True)

  result = pd.DataFrame(columns=['sentence', 'ann0','ann1','ann2'])
  result['sentence'] = ann0_df['sentence']
  result['ann0'] = ann0_df['plutchik']
  result['ann1'] = ann1_df['plutchik']
  result['ann2'] = ann2_df['plutchik']

  return result

In [9]:
train_anno = extract_3_anno(train)
test_anno = extract_3_anno(test)

In [10]:
train_anno.head(3)

,sentence,ann0,ann1,ann2
0,I began making fish curry for my boyfriend and I.,"[""joy:3"", ""trust:3"", ""surprise:3"", ""anticipati...","[""joy:2""]","[""joy:2"", ""anticipation:3""]"
1,I decided not to read a recipe since I've made...,"[""joy:2"", ""trust:2"", ""fear:3"", ""surprise:3"", ""...","[""joy:3"", ""trust:3"", ""anticipation:3""]","[""joy:2"", ""trust:3"", ""anticipation:2""]"
2,I let the curry sit before tasting.,"[""fear:2"", ""anticipation:2""]","[""joy:2"", ""trust:3""]","[""joy:3"", ""trust:2"", ""surprise:2"", ""anticipati..."


In [11]:
test_anno.head(3)

,sentence,ann0,ann1,ann2
0,He dropped one on the floor by accident.,"[""fear:2"", ""surprise:2"", ""anticipation:2""]","[""disgust:2"", ""anger:3""]","[""surprise:2"", ""anger:2""]"
1,As a joke he pretended that it was a soccer ball.,"[""joy:2""]","[""joy:3"", ""trust:3""]",[]
2,He kicked the orange across the kitchen.,"[""joy:3"", ""trust:3"", ""surprise:3"", ""anticipati...","[""disgust:3"", ""anger:3""]","[""joy:3"", ""anticipation:2""]"


### 1.2 2명 이상의 annotator가 태깅한 감정들 추출
- annotator간의 일치도 평가를 위해 2명 이상의 annotator가 태깅한 감정들만 추출
- 3명 중 2명 이상의 같은 결과는 kappa score 0.4 이상으로 판단할 수 있음

In [12]:
import re

def anno_over_2_emotion(df):
  idx = []
  emotion = []

  for i in range(0,len(df)):
    a0 = re.sub('[^a-zA,]','',df['ann0'][i]).strip()
    a0 = a0.split(sep=',')
    a1 = re.sub('[^a-zA,]','',df['ann1'][i]).strip()
    a1 = a1.split(sep=',')
    a2 = re.sub('[^a-zA,]','',df['ann2'][i]).strip()
    a2 = a2.split(sep=',')
    
    intersection = list(set(a0) & set(a1))
    intersection2 = list(set(a1) & set(a2))
    intersection3 = list(set(a0) & set(a2))

    union = list(set().union(intersection,intersection2,intersection3))
    if len(union) > 0 :
      idx.append(i)
      emotion.append(union)

  df = df.loc[idx]

  result = pd.DataFrame(columns=['sentence', 'label'])
  result['sentence'] = df['sentence']
  result['label'] = emotion
  result = result.reset_index(drop=True)

  return result

In [13]:
train_emotion = anno_over_2_emotion(train_anno)
test_emotion = anno_over_2_emotion(test_anno)

In [14]:
train_emotion.head(3)

,sentence,label
0,I began making fish curry for my boyfriend and I.,"[anticipation, joy]"
1,I decided not to read a recipe since I've made...,"[trust, anticipation, joy]"
2,I let the curry sit before tasting.,"[trust, anticipation, joy]"


In [15]:
test_emotion.head(3)

,sentence,label
0,He dropped one on the floor by accident.,"[anger, surprise]"
1,As a joke he pretended that it was a soccer ball.,[joy]
2,He kicked the orange across the kitchen.,"[anticipation, joy]"


### 1.3 emotion list를 multi label classification 형태로 변환
- label_cols = ['joy', 'anticipation', 'fear', 'surprise', 'anger', 'sadness','trust','disgust']
- [joy, anticipation] -> [1,1,0,0,0,0,0,0]

In [16]:
label_cols = ['joy', 'anticipation', 'fear', 'surprise', 'anger', 'sadness','trust','disgust']

In [17]:
def change_multi_label(df):
  sentence = []
  joy = []
  anticipation = []
  fear = []
  surprise = []
  anger = []
  sadness = []
  trust = []
  disgust = []

  for x in range(0,len(df)):
    sentence.append(df['sentence'][x])
    if 'joy' in df['label'][x]:
      joy.append(1)
    elif len(joy) != len(sentence):
      joy.append(0)

    if 'anticipation' in df['label'][x]:
      anticipation.append(1)
    elif len(anticipation) != len(sentence):
      anticipation.append(0)
    
    if 'fear' in df['label'][x]:
      fear.append(1)
    elif len(fear) != len(sentence):
      fear.append(0)

    if 'surprise' in df['label'][x]:
      surprise.append(1)
    elif len(surprise) != len(sentence):
      surprise.append(0)
    
    if 'anger' in df['label'][x]:
      anger.append(1)
    elif len(anger) != len(sentence):
      anger.append(0)

    if 'sadness' in df['label'][x]:
      sadness.append(1)
    elif len(sadness) != len(sentence):
      sadness.append(0)

    if 'trust' in df['label'][x]:
      trust.append(1)
    elif len(trust) != len(sentence):
      trust.append(0)

    if 'disgust' in df['label'][x]:
      disgust.append(1)
    elif len(disgust) != len(sentence):
      disgust.append(0)

  result = pd.DataFrame(columns=['sentence', 'joy','anticipation','fear','surprise','anger','sadness','trust','disgust'])
  result['sentence'] = sentence
  result['joy'] = joy
  result['anticipation'] = anticipation
  result['fear'] = fear
  result['surprise'] = surprise
  result['anger'] = anger
  result['sadness'] = sadness
  result['trust'] = trust
  result['disgust'] = disgust
  result = result.drop_duplicates('sentence', keep='first')

  return result

In [18]:
df_train = change_multi_label(train_emotion)
df_test = change_multi_label(test_emotion)

In [19]:
len(df_train)

9186

In [20]:
len(df_test)

10210

In [21]:
df_all = pd.concat([df_train,df_test])

In [22]:
df_all = df_all.drop_duplicates('sentence', keep='first')
df_all = df_all.reset_index(drop=True)

In [23]:
len(df_all)

19393

In [24]:
df_all.tail()

,sentence,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
19388,The light bulb was broken.,0,1,0,1,1,0,0,0
19389,Carol had to move around in the dark.,0,1,1,1,0,0,0,0
19390,She was at a party that had amazing food set out.,1,0,0,0,0,0,1,0
19391,She had been dieting for weeks and was starving.,0,0,0,0,1,0,0,0
19392,Rosemary swore to eat healthier tomorrow.,1,1,0,0,0,0,1,0


In [25]:
df_train = pd.DataFrame(columns=['sentence', 'joy','anticipation','fear','surprise','anger','sadness','trust','disgust'])
df_test = pd.DataFrame(columns=['sentence', 'joy','anticipation','fear','surprise','anger','sadness','trust','disgust'])

In [26]:
df_train = df_all.sample(frac=0.6, random_state=123)  # 항상 같은 값 유지
df_test = df_all.drop(df_train.index)

In [27]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [28]:
print(len(df_train), len(df_test))

11636 7757


In [29]:
a = set(df_train['sentence']).intersection(set(df_test['sentence']))  # train과 test 중복 문장이 0개 임을 확인 -> 학습데이터 시험데이터 비교 코드
len(a)

0

In [30]:
df_train.head()

,sentence,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
0,Her mother called her in.,1,0,0,0,0,0,0,0
1,He ended up delivering pizzas and had a blast.,1,0,0,0,0,0,0,0
2,"He obliged and now walks three to four dogs, t...",1,1,0,0,0,0,1,0
3,She was sad not to see a single familiar face.,0,1,1,1,0,1,0,0
4,Rick rode a bus to the Opening Ceremonies with...,1,1,0,1,0,0,1,0


In [31]:
df_test.head()

,sentence,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
0,I began making fish curry for my boyfriend and I.,1,1,0,0,0,0,0,0
1,I decided not to read a recipe since I've made...,1,1,0,0,0,0,1,0
2,Jervis has been single for a long time.,0,0,0,0,0,1,0,1
3,The radio played song after song all night lon...,1,0,0,0,0,0,0,0
4,He was the best barista in the whole town.,1,0,0,0,0,0,0,0


### 1.4 predict 결과를 저장할 df_result 생성
- 모든 감정은 0.5로 초기화
- test 셋과 동일한 sentence

In [32]:
df_result = df_test
df_result = df_result.drop_duplicates('sentence', keep='first')

df_result[label_cols] = 0.5
df_result.set_index('sentence',inplace=True)

In [33]:
df_result.head()

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
sentence,,,,,,,,
I began making fish curry for my boyfriend and I.,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
I decided not to read a recipe since I've made so many in my life.,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
Jervis has been single for a long time.,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
The radio played song after song all night long as Ray slumbered.,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
He was the best barista in the whole town.,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [34]:
len(df_result)

7757

## 2. Preprocessing (Tokenization, Truncation & Padding)

In [35]:
!pip install transformers==2.3.0

     |████████████████████████████████| 450kB 11.9MB/s 
     |████████████████████████████████| 1.1MB 54.2MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
     |████████████████████████████████| 7.0MB 48.1MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for boto3: filename=boto3-1.16.33-py2.py3-none-any.whl size=128451 sha256=224116de24bc97d6b4f48fdb9970b6f37ded4eecc7e41bdf5551a196e9c23a39
  Stored in directory: /root/.cache/pip/wheels/28/c0/ba/5f9cce3a8686eea945fd594ae8fcdec24b1bd7402d3d065a47
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=883798e2d93d14b17cb6e45167d84accb62ea0ba584322d20ab8fa614a52ffad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built boto3 sacremoses
ERROR: botocore 1.19.33 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll h

In [36]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [37]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 64

def tokenize_sentences(sentences, tokenizer, max_seq_len = 64):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(df_train['sentence'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

In [38]:
from sklearn.model_selection import train_test_split

labels =  df_train[label_cols].values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.3)

train_size = len(train_inputs)
validation_size = len(validation_inputs)

print(train_size, validation_size)

8145 3491


In [39]:
BATCH_SIZE = 32
NR_EPOCHS = 10

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [40]:
df_test_labels = df_test[label_cols].values

test_input_ids = tokenize_sentences(df_test['sentence'], tokenizer, MAX_LEN)
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

In [41]:
TEST_BATCH_SIZE = 32
test_steps = len(df_test) // TEST_BATCH_SIZE

test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False, epochs=1)
print(len(test_input_ids))

7757


## 3. BERT Model

In [42]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols))

In [43]:
model(train_inputs[:1],train_masks[:1]).numpy()
model.summary()

Model: "bert_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model (TFBertModel)  multiple                  109482240 
_________________________________________________________________
dense (Dense)                multiple                  6152      
Total params: 109,488,392
Trainable params: 109,488,392
Non-trainable params: 0
_________________________________________________________________


## 4. Training Loop

In [44]:
# import time
# from transformers import create_optimizer

# steps_per_epoch = train_size // BATCH_SIZE
# validation_steps = validation_size // BATCH_SIZE

# # | Loss Function
# loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# train_loss = tf.keras.metrics.Mean(name='train_loss')
# validation_loss = tf.keras.metrics.Mean(name='test_loss')

# # | Optimizer (with 1-cycle-policy)
# warmup_steps = steps_per_epoch // 3
# total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
# optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# # | Metrics
# train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
# validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

# @tf.function
# def train_step(model, token_ids, masks, labels):
#     labels = tf.dtypes.cast(labels, tf.float32)

#     with tf.GradientTape() as tape:
#         predictions = model(token_ids, attention_mask=masks)
#         loss = loss_object(labels, predictions)

#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables), 1.0)

#     train_loss(loss)

#     for i, auc in enumerate(train_auc_metrics):
#         auc.update_state(labels[:,i], predictions[:,i])
        
# @tf.function
# def validation_step(model, token_ids, masks, labels):
#     labels = tf.dtypes.cast(labels, tf.float32)

#     predictions = model(token_ids, attention_mask=masks, training=False)
#     v_loss = loss_object(labels, predictions)

#     validation_loss(v_loss)
#     for i, auc in enumerate(validation_auc_metrics):
#         auc.update_state(labels[:,i], predictions[:,i])
                                              
# def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
#     for epoch in range(epochs):
#         print('=' * 50, f"EPOCH {epoch+1}", '=' * 50)

#         start = time.time()

#         for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
#             train_step(model, token_ids, masks, labels)
#             if i % 1000 == 0:
#                 print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
#                 for i, label_name in enumerate(label_cols):
#                     print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
#                     train_auc_metrics[i].reset_states()
        
#         for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
#             validation_step(model, token_ids, masks, labels)

#         print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

#         for i, label_name in enumerate(label_cols):
#             print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
#             validation_auc_metrics[i].reset_states()

#         print('\n')

        
# train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

In [45]:
#model.save_weights("multi_emotion_classification.h5")

## 5. 사전 학습한 모델 로드 및 결과 확인

In [46]:
!gdown --id 1P9_JJ0mkwGC_eiZzh3nfIfwKlQFMpA53  # 사전 학습 모델 다운로드 코드

Downloading...
From: https://drive.google.com/uc?id=1P9_JJ0mkwGC_eiZzh3nfIfwKlQFMpA53
To: /content/multi_emotion_classification.h5
438MB [00:02, 147MB/s]


In [47]:
model.load_weights('/content/multi_emotion_classification.h5')  # 사전 학습 모델 적용 코드

In [48]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report,hamming_loss, f1_score

In [49]:
test_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
    labels = df_test_labels[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]
    labels = tf.dtypes.cast(labels, tf.float32)
    
    sample_ids = df_test.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['sentence']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_result.loc[sample_ids, label_cols] = predictions

    for i, auc in enumerate(test_auc_metrics):
      auc.update_state(labels[:,i], predictions[:,i])

In [50]:
for i, label_name in enumerate(label_cols):
    print(f"{label_name} roc_auc {test_auc_metrics[i].result()}")
    test_auc_metrics[i].reset_states()

joy roc_auc 0.900753378868103
anticipation roc_auc 0.6956520676612854
fear roc_auc 0.8043549060821533
surprise roc_auc 0.7219706773757935
anger roc_auc 0.840342104434967
sadness roc_auc 0.8476757407188416
trust roc_auc 0.7645024657249451
disgust roc_auc 0.825882613658905


In [51]:
df_result.head()

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
sentence,,,,,,,,
I began making fish curry for my boyfriend and I.,0.981032,0.717183,0.017992,0.216353,0.012455,0.027910,0.561285,0.020403
I decided not to read a recipe since I've made so many in my life.,0.421317,0.483951,0.117625,0.424697,0.084600,0.098903,0.123876,0.063185
Jervis has been single for a long time.,0.878109,0.517616,0.064829,0.074441,0.020581,0.051312,0.483384,0.015689
The radio played song after song all night long as Ray slumbered.,0.569796,0.393155,0.094380,0.155930,0.080234,0.192368,0.212913,0.062805
He was the best barista in the whole town.,0.966941,0.584770,0.027806,0.207532,0.010264,0.015550,0.568414,0.010594


In [52]:
y_true = df_test_labels
y_pred = df_result[label_cols].values

In [53]:
thresh = 0.5
y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in y_pred])

In [54]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[3086,  699],
        [ 664, 3308]],

       [[1881, 1539],
        [1221, 3116]],

       [[5249,  557],
        [1082,  869]],

       [[3741,  900],
        [1578, 1538]],

       [[5812,  469],
        [ 825,  651]],

       [[4998,  667],
        [ 854, 1238]],

       [[4558,  907],
        [1244, 1048]],

       [[6099,  361],
        [ 902,  395]]])

In [55]:
print(f1_score(y_true, y_pred,average=None))  # 각 감정별 f1-score : joy	anticipation	fear	surprise	anger	sadness	trust	disgust

[0.82917659 0.6930605  0.51465798 0.55383507 0.50154083 0.6194646
 0.49352484 0.38480273]


In [56]:
print(f1_score(y_true, y_pred,average='micro')) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

0.6270395669545045


--------------

In [57]:
def tokenize_sentences_single(sentence, tokenizer, max_seq_len = 64):
    tokenized_sentence = []
    tokenized = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                    )
        
    tokenized_sentence.append(tokenized)

    return tokenized_sentence

def prediction_sample(story):
  result = pd.DataFrame(columns=['joy','anticipation','fear','surprise','anger','sadness','trust','disgust'])

  sample_input_ids = tokenize_sentences_single(story, tokenizer, MAX_LEN)
  sample_input_ids = pad_sequences(sample_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  sample_attention_masks = create_attention_masks(sample_input_ids)

  predictions = model(sample_input_ids, attention_mask=sample_attention_masks).numpy()

  y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in predictions])
  result.loc[story, label_cols] = y_pred

  return result 

In [58]:
prediction_sample('Jennifer has a big exam tomorrow. She got so stressed, she pulled an all-nighter.')  # 문장 별 다중 emotion 분류 실행 코드

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
"Jennifer has a big exam tomorrow. She got so stressed, she pulled an all-nighter.",0,0,1,0,0,1,0,0


In [59]:
prediction_sample('She went into class the next day, weary as can be.') 

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
"She went into class the next day, weary as can be.",0,0,0,0,0,1,0,0


In [60]:
prediction_sample('Her teacher stated that the test is postponed for next week.') 

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
Her teacher stated that the test is postponed for next week.,0,1,1,1,1,1,0,0


In [61]:
prediction_sample('Jennifer felt bittersweet about it.') 

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
Jennifer felt bittersweet about it.,0,0,0,0,0,1,0,0


In [62]:
prediction_sample('Her roommate asked her to go to a nearby city for a concert.') 

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
Her roommate asked her to go to a nearby city for a concert.,1,1,0,0,0,0,0,0


In [63]:
prediction_sample('Karen agreed happily. The show was absolutely exhilarating.') 

,joy,anticipation,fear,surprise,anger,sadness,trust,disgust
Karen agreed happily. The show was absolutely exhilarating.,1,1,0,1,0,0,0,0


----------------------